In [ ]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

In [ ]:
# 查看当前kernerl下的package
!pip list --format=columns

# 训练

In [1]:
# -*- coding:utf-8 -*-

import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import tensorflow as tf

from tensorflow.contrib import rnn

import tensorflow.contrib.layers as layers
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

import tensorflow as tf

import pandas as pd
import numpy as np
from gensim.models import Word2Vec





class Settings(object):

    def __init__(self):


        self.model_name = 'wd_1_2_cnn_max'
        self.query_len = 20
        self.title_len = 20
        self.filter_sizes = [2, 3, 4, 5]
        self.n_filter = 256
        self.fc_hidden_size = 1024
        self.n_class = 1
        self.train_batch_size=512
        self.vaild_batch_size=512
        self.lr=0.0025
        self.decay_step=20000
        self.decay_rate=0.8
        self.w=0





class TEXTCNN(object):

    def __init__(self,  settings):


        self.model_name = settings.model_name
        self.query_len = settings.query_len
        self.title_len = settings.title_len
        self.filter_sizes = settings.filter_sizes
        self.n_filter = settings.n_filter
        self.n_filter_total = self.n_filter * len(self.filter_sizes)
        self.n_class = settings.n_class
        self.fc_hidden_size = settings.fc_hidden_size
        self._global_step = tf.Variable(0, trainable=False, name='Global_Step')
        self.update_emas = list()
        # placeholders
        self.training = tf.placeholder(tf.bool)
        self._keep_prob = tf.placeholder(tf.float32, [])
        self._batch_size = tf.placeholder(tf.int32, [])
        self.embedding_size=100
        self.w=0

        with tf.name_scope('Inputs'):
            self.embedded_x_query=tf.placeholder(tf.float32,[None,self.query_len,self.embedding_size])#h r
            self.embedded_x_title=tf.placeholder(tf.float32,[None,self.title_len,self.embedding_size])# r
            self._y_inputs = tf.placeholder(tf.float32, [None, self.n_class], name='y_input')

        output_title = self.cnn_inference(self.embedded_x_query, self.query_len,'query')
        output_title = tf.expand_dims(output_title, 0)

        output_content = self.cnn_inference(self.embedded_x_title, self.title_len,'title')
        output_content = tf.expand_dims(output_content, 0)


        with tf.variable_scope('fc-bn-layer'):
            output = tf.concat([output_title, output_content], axis=0)
            output = tf.reduce_max(output, axis=0)
            W_fc = self.weight_variable([self.n_filter_total, self.fc_hidden_size], name='Weight_fc')
            h_fc = tf.matmul(output, W_fc, name='h_fc')
            fc_bn=tf.layers.batch_normalization(h_fc,training=self.training)
            # self.update_emas.append(update_ema_fc)
            self.fc_bn_relu = tf.nn.relu(fc_bn, name="relu")
            fc_bn_drop = tf.nn.dropout(self.fc_bn_relu, self.keep_prob)

        with tf.variable_scope('out_layer'):
            W_out = self.weight_variable([self.fc_hidden_size, self.n_class], name='Weight_out')
            b_out = self.bias_variable([self.n_class], name='bias_out')
            self.sim = self.lcnn_foward()
            self._y_pred = tf.nn.xw_plus_b(fc_bn_drop, W_out, b_out, name='y_pred')  # 每个类别的分数 scores
            self._y_pred = self._y_pred+self.sim 
        

        with tf.name_scope('loss'):
            self._loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=self._y_pred, labels=self._y_inputs))

    @property
    def tst(self):
        return self._tst

    @property
    def keep_prob(self):
        return self._keep_prob

    @property
    def batch_size(self):
        return self._batch_size

    @property
    def global_step(self):
        return self._global_step

    @property
    def X1_inputs(self):
        return self._X1_inputs

    @property
    def X2_inputs(self):
        return self._X2_inputs

    @property
    def y_inputs(self):
        return self._y_inputs

    @property
    def y_pred(self):
        return self._y_pred

    @property
    def loss(self):
        return self._loss

    def weight_variable(self, shape, name):
        """Create a weight variable with appropriate initialization."""
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial, name=name)

    def bias_variable(self, shape, name):
        """Create a bias variable with appropriate initialization."""
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial, name=name)


    def lcnn_foward(self):
        with tf.name_scope('sematic_matching'):
            A = self.make_word_sim(self.embedded_x_query, self.embedded_x_title)
            # A = A * self.make_attention_mat(self.embedded_x_query, self.embedded_x_title)
            left_hat = self.semantic_match(self.embedded_x_title, A, self.w, scope='left', type='global')
            right_hat = self.semantic_match(self.embedded_x_query, tf.transpose(A, [0, 2, 1]), self.w, scope='right', type='global')

        with tf.name_scope('Decomposition'):
            left_pos, left_neg = self.decompose(self.embedded_x_query, left_hat, scope='left', type='linear')
            right_pos, right_neg = self.decompose(self.embedded_x_title, right_hat, scope='right', type='linear')
            left_decomp = tf.expand_dims(tf.concat([left_pos, left_neg], axis=-1), -1)
            right_decomp = tf.expand_dims(tf.concat([right_pos, right_neg], axis=-1), -1)



        with tf.name_scope('Composition'):
            sims = []
            # pooled_left = []
            # pooled_right = []
            for i, filter_size in enumerate(self.filter_sizes):
                left_conv = self.convolution(left_decomp, 2 * self.embedding_size, filter_size, 70, 'conv_left')
                left_pooled = self.max_pool(left_conv, filter_size, 'max_pooling_left')
                left_pooled_flatten = tf.reshape(left_pooled, [-1, 70], name='left_pooled_flatten')

                right_conv = self.convolution(right_decomp, 2 * self.embedding_size, filter_size, 70, 'conv_right')
                right_pooled = self.max_pool(right_conv, filter_size, 'max_pooling_right')
                right_pooled_flatten = tf.reshape(right_pooled, [-1, 70], name='right_pooled_flatten')
                
                sims.append(self.get_sim(left_pooled_flatten, right_pooled_flatten, 70, str(filter_size)))
                sims.append(self.get_cos_sim(left_pooled_flatten, right_pooled_flatten, str(filter_size)))
                # pooled_left.append(left_pooled_flatten)
                # pooled_right.append(right_pooled_flatten)

            sims = tf.concat(sims, axis=-1)
            # pooled_left = tf.concat(pooled_left, axis=-1)
            # pooled_right = tf.concat(pooled_right, axis=-1)


        with tf.variable_scope('output_layer'):
            self.output_features = sims
            self.output_features = tf.layers.batch_normalization(self.output_features,training=self.training,name='fea_bn_')
            self.output_features = tf.nn.dropout(self.output_features, self.keep_prob, name='hidden_output_drop')
            self.fc = tf.contrib.layers.fully_connected(
                inputs = self.output_features,
                num_outputs= self.fc_hidden_size,
                weights_initializer=tf.contrib.layers.xavier_initializer(),
                biases_initializer=tf.constant_initializer(1e-02),
                scope="FC"
            )
            self.fc_bn = tf.layers.batch_normalization(self.fc,training=self.training,name='fc_bn_')
            self.fc_bn = tf.nn.relu(self.fc_bn)
            self.hidden_drop = tf.nn.dropout(self.fc_bn, self.keep_prob, name='fc_drop')

        
        with tf.variable_scope('out_layer'):
            sim = tf.contrib.layers.fully_connected(
                inputs = self.hidden_drop,
                num_outputs= self.n_class,
                activation_fn = None,
                weights_initializer=tf.contrib.layers.xavier_initializer(),
                biases_initializer=tf.constant_initializer(1e-04),
                scope="FC_2"
            )
        return sim



    def get_cos_sim(self, x1, x2, name_scope):
        with tf.variable_scope('cos_sim_' + name_scope):
            norm1 = tf.sqrt(tf.reduce_sum(tf.square(x1), axis=1))
            norm2 = tf.sqrt(tf.reduce_sum(tf.square(x2), axis=1))
            dot_products = tf.reduce_sum(x1 * x2, axis=1, name='cos_sim')

            cos_sim = tf.expand_dims(dot_products / (norm1 * norm2), -1)
            return cos_sim


    def get_sim(self, x1, x2, d, name_scope):
        with tf.variable_scope("similarity_" + name_scope):
            M = tf.get_variable(
                name = 'M',
                shape = [d, d],
                initializer = tf.contrib.layers.xavier_initializer()
            )

            x1_trans = tf.matmul(x1, M)
            sims = tf.reduce_sum(tf.multiply(x1_trans, x2), axis=1, keepdims=True)
            return sims

    def max_pool(self, x, filter_size, name_scope):
        with tf.name_scope(name_scope + str(filter_size)):
            pool_width = 20 - filter_size + 1

            pooled = tf.layers.max_pooling2d(
                inputs=x,
                pool_size=(pool_width, 1),
                strides=1,
                padding='VALID',
            )
            # [batch_size, 1, 1, n_filter]
            return pooled

    def convolution(self, x, d, filter_size, n_filter, name_scope):
        with tf.name_scope(name_scope + str(filter_size)):
            conv = tf.contrib.layers.conv2d(
                inputs=x,
                num_outputs=n_filter,
                kernel_size=(filter_size, d),
                stride=1,
                padding='VALID',
                activation_fn=tf.nn.relu,
                weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
                biases_initializer=tf.constant_initializer(0.1),
                trainable=True
            )
            # [batch_size, |s| - filter_size + 1, 1, n_filter]
            return conv

    def decompose(self, X, X_hat, scope, type='linear'):
        with tf.variable_scope('decompose_'+scope):
            if type == 'rigid':
                mask_pos = tf.cast(tf.equal(X, X_hat), dtype='float32')
                X_pos = X * mask_pos
                mask_neg = tf.cast(tf.equal(mask_pos, 0.0), dtype='float32')
                X_neg = X * mask_neg
            elif type == 'linear':
                denom = (tf.linalg.norm(X, axis=-1, keepdims=True)) * (tf.linalg.norm(X_hat, axis=-1, keepdims=True))
                alpha = tf.reduce_sum(X * X_hat, axis=-1, keepdims=True) / denom
                X_pos = alpha * X
                X_neg = (1 - alpha) * X
            elif type == 'orthogonal':
                denom = tf.reduce_sum(X_hat * X_hat, axis=-1, keepdims=True)
                X_pos = tf.reduce_sum(X * X_hat, axis=-1, keepdims=True) / denom  * X_hat
                X_neg = X - X_pos
            return X_pos, X_neg

    def semantic_match(self, X, A, w, scope, type='local_w'):
        # x: batch * length2 * embedding_size
        # A: batch * length1 * length2
        # return: batch * length * embedding_size
        # type == 'max'只需将w设置为0， 'global'只需将w设置为sentence_length
        with tf.variable_scope('semantic_' + scope):
            pivot = tf.argmax(A, axis=-1)
            if type == 'global':
                return tf.matmul(A, X)/tf.expand_dims(tf.reduce_sum(A, axis=-1), axis=-1)
            elif type == 'local_w':
                upper = tf.expand_dims(tf.minimum(pivot + w, 20),-1)
                lower = tf.expand_dims(tf.maximum(pivot - w, 0), -1)
                indices = tf.tile(tf.expand_dims(tf.range(start=0, limit=20, dtype='int64'), axis=0), [50, 1])
                print(indices,"*********************",lower)
                mask = tf.cast(((indices>=lower)&(indices<=upper)), 'float32')
                return tf.matmul(A*mask, X)/tf.expand_dims(tf.reduce_sum(A*mask, axis=-1), axis=-1)

    def make_word_sim(self, x, y):
        dot = tf.einsum('abd,acd->abc', x, y)
        norm1 = tf.sqrt(tf.reduce_sum(tf.square(x), axis=2, keepdims=True))
        norm2 = tf.sqrt(tf.reduce_sum(tf.square(y), axis=2, keepdims=True))
        return dot / tf.einsum('abd,acd->abc', norm1, norm2)
    


    def cnn_inference(self, X_inputs, n_step,name):
        """TextCNN 模型。
        Args:
            X_inputs: tensor.shape=(batch_size, n_step)
        Returns:
            title_outputs: tensor.shape=(batch_size, self.n_filter_total)
        """
        inputs = X_inputs
        inputs = tf.expand_dims(inputs, -1)
        pooled_outputs = list()
        for i, filter_size in enumerate(self.filter_sizes):
            with tf.variable_scope(("conv-maxpool-%s" % filter_size)+name):
                # Convolution Layer
                filter_shape = [filter_size, self.embedding_size, 1, self.n_filter]
                W_filter = self.weight_variable(shape=filter_shape, name='W_filter')
                beta = self.bias_variable(shape=[self.n_filter], name='beta_filter')
                conv = tf.nn.conv2d(inputs, W_filter, strides=[1, 1, 1, 1], padding="VALID", name="conv")
                conv_bn=tf.layers.batch_normalization(conv,training=self.training,name=name)
                # Apply nonlinearity, batch norm scaling is not useful with relus
                # batch norm offsets are used instead of biases,使用 BN 层的 offset，不要 biases
                h = tf.nn.relu(conv_bn, name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(h, ksize=[1, n_step - filter_size + 1, 1, 1],
                                        strides=[1, 1, 1, 1], padding='VALID', name="pool")
                pooled_outputs.append(pooled)
        h_pool = tf.concat(pooled_outputs, 3)
        h_pool_flat = tf.reshape(h_pool, [-1, self.n_filter_total])
        return h_pool_flat  # shape = [batch_size, self.n_filter_total]



def loader(data_store_path=''):############
    # all_data=pd.read_csv(data_store_path+"data_sets/train_feature_second_no_query_and_title_data.csv")
    # query_title=pd.read_csv(data_store_path+"data_sets/train_data.csv",usecols=['query','title'])
    print("开始加载数据")
    all_data=pd.read_pickle(data_store_path+"/home/kesci/work/second_zzp/090-095.pickle")[0:20000000]
    data_split=StratifiedShuffleSplit(n_splits=2,test_size=0.001,random_state=1)
    train_index,vaild_index=data_split.split(all_data['label'],all_data['label']).__next__()
    train_data=all_data.iloc[train_index]
    vaild_data=all_data.iloc[vaild_index]
    print("加载数据完成")
    return train_data,vaild_data

def batch_iter(data_x,data_y, batch_size, num_epochs, shuffle=True):#这个就是产生batch的，可以直接用
    """
    Generates a batch iterator for a dataset.
    """
    # data_x=data_x.tolist()
    data=list(zip(data_x,data_y))
    data=np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:#如果为true 则代表允许随机取数据
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]#随机取数据
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]#而不用一个个加到列表里了。这是一个batch，嗯哼
            #取到的维度为（batchsize,2） y为(batchsize,1) 里面存的是序号

from gensim import models
import pickle
fw=open('dictionary/dictionary.pickle','rb')
dictionary=pickle.load(fw)
import pickle
fw=open('tfidf_model/tfidf_model.pickle','rb')
tfidf_model=pickle.load(fw)

def get_w2v_array(word_list,max_len):
    single_sentence_tfidf=tfidf_model[dictionary.doc2bow(word_list)]
    # single_sentence_tfidf={dictionary[word_id_tfidf[0]]:word_id_tfidf[1] for word_id_tfidf in single_sentence_tfidf}
    array = np.zeros((max_len, 100))
    if len(word_list)<=max_len:
        for i in range(len(word_list)):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
                # try:
                #     array[i][:]*=single_sentence_tfidf[str(word_list[i])]

                # except:
                #     pass
                
    else:
        for i in range(max_len):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
                # try:
                #     array[i][:]*=single_sentence_tfidf[str(word_list[i])]
                # except:
                #     pass
    return array

settings = Settings()

train_batches,vaild_data=loader()

train_batches=np.array(train_batches)
train_batches_y=train_batches[:,2]
train_batches_x=np.delete(train_batches, 2, axis=1)######这里是除了label以外的维数


batches=batch_iter(train_batches_x,train_batches_y,settings.train_batch_size,4)

vaild_batches=np.array(vaild_data)
vaild_batches_y=vaild_batches[:,2]
vaild_batches_x=np.delete(vaild_batches, 2, axis=1)
vaild_data=batch_iter(vaild_batches_x,vaild_batches_y,settings.vaild_batch_size,1000)

print("train starting")
w2v_model = Word2Vec.load('w2v_model3/w2v_all_data_model.txt')
word_wv= w2v_model.wv


settings = Settings()

settings.all_features_num=train_batches_x.shape[1]-2###########自动设置数值型特征数量
model = TEXTCNN( settings)

learning_rate = tf.train.exponential_decay(settings.lr, model.global_step, settings.decay_step,
                                       settings.decay_rate, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate)

update_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    grads_and_vars=optimizer.compute_gradients(model.loss)
    train_op=optimizer.apply_gradients(grads_and_vars,global_step=model.global_step)

config = tf.ConfigProto() 
config.gpu_options.allow_growth = True 
sess = tf.Session(config=config) #启动创建的模型
# sess.run(tf.initialize_all_variables())

# saver = tf.train.Saver(tf.global_variables())
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
saver.restore(sess, tf.train.latest_checkpoint('/home/kesci/work/tf_model_textcnn_lcnn/'))
per_train_loss_avg_list=[]
per_vaild_loss_avg_list=[]
print("start")
parts_time=[]
for batch in batches:
    from time import time
    time1=time()
    batch_x,batch_y=zip(*batch)
    batch_y=np.array(batch_y)[:,None]
    batch_x=[i.tolist() for i in batch_x]
    batch_x=pd.DataFrame(batch_x)

    batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
    batch_x=np.array(batch_x)
    #####[None,title_len,embedding_size]
    #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
    #query   title   title_count         title_ctr      label
    batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=batch_x)
    batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=batch_x)

        
    #     train_op=optimizer.apply_gradients(grads_and_vars,global_step=global_step)                              
    feed_dict = {model.embedded_x_query: batch_x_query,
                 model.embedded_x_title: batch_x_title,
                 model.y_inputs: batch_y,
                 model.batch_size: batch_x.shape[0], 
                 model.training: True, 
                 model.keep_prob: 0.8}
    
    _,step,loss_out=sess.run([train_op,model.global_step,model.loss],feed_dict)
    time2=time()
    parts_time.append(time2-time1)
    if step%10==0:
        print("step {},loss:{:g}".format(step,loss_out))              
    per_train_loss_avg_list.append(loss_out)
    
    if step%(int(50000000/1004))==0:
        print("batch ended")
    if step%500==0:
        time_single_part=sum(parts_time)
        ed_time=(50000000/settings.train_batch_size-step)/500*time_single_part
        print("距离训练完一整轮剩余时间：",ed_time/60," 分钟")
        parts_time=[]
        vaild_x,vaild_y=zip(*(vaild_data.__next__()))
        vaild_y=np.array(vaild_y)[:,None]
        vaild_x=[i.tolist() for i in vaild_x]
        vaild_x=pd.DataFrame(vaild_x)
        vaild_x[[0,1]]=vaild_x[[0,1]].applymap(lambda x:x.split(' '))
        vaild_x=np.array(vaild_x)
    
        #####[None,title_len,embedding_size]
        #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
        #query   title   title_count         title_ctr      label
        vaild_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=vaild_x)
        vaild_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=vaild_x)


        # #########################这是连续型特征
        # vaild_x_ctr_fea=vaild_x[:,2:]######这里需要到时候调整，按label的位置和特征位置
        # vaild_x_ctr_fea=np.delete(vaild_x_ctr_fea, -1, axis=1)######需要把类别型特征去掉
        # ##########################
        # ###########################这是类别型特征
        # vaild_x_tfidf=vaild_x[:,-1][:,None]
        # #########################################


        feed_dict = {model.embedded_x_query: vaild_x_query,
                     model.embedded_x_title: vaild_x_title,
                     model.y_inputs: vaild_y,
                     model.batch_size: vaild_x.shape[0], 
                     model.training: False, 
                     model.keep_prob: 1}
        
        
        step,loss_out=sess.run([model.global_step,model.loss],feed_dict)
        per_vaild_loss_avg_list.append(loss_out)
        per_train_loss_avg=sum(per_train_loss_avg_list)/len(per_train_loss_avg_list)
        print("per_train_loss_avg:",per_train_loss_avg,"**********")
        per_train_loss_avg_list=[]
        print("vaild_loss:",loss_out)
    if step%5000==0:
        per_vaild_loss_avg=sum(per_vaild_loss_avg_list)/len(per_vaild_loss_avg_list)
        per_vaild_loss_avg_list=[]
        print("per_vaild_loss_avg:",per_vaild_loss_avg,"************")
        saver = tf.train.Saver(max_to_keep = 3,var_list=tf.global_variables())
        saver = saver.save(sess, "/home/kesci/work/tf_model_textcnn_lcnn/cnn", global_step=step)

# 预测

In [1]:
# -*- coding:utf-8 -*-

import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import tensorflow as tf

from tensorflow.contrib import rnn

import tensorflow.contrib.layers as layers
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

import tensorflow as tf

import pandas as pd
import numpy as np
from gensim.models import Word2Vec






class Settings(object):

    def __init__(self):


        self.model_name = 'wd_1_2_cnn_max'
        self.query_len = 20
        self.title_len = 20
        self.filter_sizes = [2, 3, 4, 5]
        self.n_filter = 256
        self.fc_hidden_size = 1024
        self.n_class = 1
        self.train_batch_size=1024
        self.vaild_batch_size=512
        self.lr=0.0025
        self.decay_step=20000
        self.decay_rate=0.8
        self.w=0





class TEXTCNN(object):

    def __init__(self,  settings):


        self.model_name = settings.model_name
        self.query_len = settings.query_len
        self.title_len = settings.title_len
        self.filter_sizes = settings.filter_sizes
        self.n_filter = settings.n_filter
        self.n_filter_total = self.n_filter * len(self.filter_sizes)
        self.n_class = settings.n_class
        self.fc_hidden_size = settings.fc_hidden_size
        self._global_step = tf.Variable(0, trainable=False, name='Global_Step')
        self.update_emas = list()
        # placeholders
        self.training = tf.placeholder(tf.bool)
        self._keep_prob = tf.placeholder(tf.float32, [])
        self._batch_size = tf.placeholder(tf.int32, [])
        self.embedding_size=100
        self.w=0

        with tf.name_scope('Inputs'):
            self.embedded_x_query=tf.placeholder(tf.float32,[None,self.query_len,self.embedding_size])#h r
            self.embedded_x_title=tf.placeholder(tf.float32,[None,self.title_len,self.embedding_size])# r
            self._y_inputs = tf.placeholder(tf.float32, [None, self.n_class], name='y_input')

        output_title = self.cnn_inference(self.embedded_x_query, self.query_len,'query')
        output_title = tf.expand_dims(output_title, 0)

        output_content = self.cnn_inference(self.embedded_x_title, self.title_len,'title')
        output_content = tf.expand_dims(output_content, 0)


        with tf.variable_scope('fc-bn-layer'):
            output = tf.concat([output_title, output_content], axis=0)
            output = tf.reduce_max(output, axis=0)
            W_fc = self.weight_variable([self.n_filter_total, self.fc_hidden_size], name='Weight_fc')
            h_fc = tf.matmul(output, W_fc, name='h_fc')
            fc_bn=tf.layers.batch_normalization(h_fc,training=self.training)
            # self.update_emas.append(update_ema_fc)
            self.fc_bn_relu = tf.nn.relu(fc_bn, name="relu")
            fc_bn_drop = tf.nn.dropout(self.fc_bn_relu, self.keep_prob)

        with tf.variable_scope('out_layer'):
            W_out = self.weight_variable([self.fc_hidden_size, self.n_class], name='Weight_out')
            b_out = self.bias_variable([self.n_class], name='bias_out')
            self.sim = self.lcnn_foward()
            self._y_pred = tf.nn.xw_plus_b(fc_bn_drop, W_out, b_out, name='y_pred')  # 每个类别的分数 scores
            self._y_pred = self._y_pred+self.sim 
        

        with tf.name_scope('loss'):
            self._loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=self._y_pred, labels=self._y_inputs))
            self._y_pred = tf.sigmoid(self._y_pred)

    @property
    def tst(self):
        return self._tst

    @property
    def keep_prob(self):
        return self._keep_prob

    @property
    def batch_size(self):
        return self._batch_size

    @property
    def global_step(self):
        return self._global_step

    @property
    def X1_inputs(self):
        return self._X1_inputs

    @property
    def X2_inputs(self):
        return self._X2_inputs

    @property
    def y_inputs(self):
        return self._y_inputs

    @property
    def y_pred(self):
        return self._y_pred

    @property
    def loss(self):
        return self._loss

    def weight_variable(self, shape, name):
        """Create a weight variable with appropriate initialization."""
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial, name=name)

    def bias_variable(self, shape, name):
        """Create a bias variable with appropriate initialization."""
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial, name=name)


    def lcnn_foward(self):
        with tf.name_scope('sematic_matching'):
            A = self.make_word_sim(self.embedded_x_query, self.embedded_x_title)
            # A = A * self.make_attention_mat(self.embedded_x_query, self.embedded_x_title)
            left_hat = self.semantic_match(self.embedded_x_title, A, self.w, scope='left', type='global')
            right_hat = self.semantic_match(self.embedded_x_query, tf.transpose(A, [0, 2, 1]), self.w, scope='right', type='global')

        with tf.name_scope('Decomposition'):
            left_pos, left_neg = self.decompose(self.embedded_x_query, left_hat, scope='left', type='linear')
            right_pos, right_neg = self.decompose(self.embedded_x_title, right_hat, scope='right', type='linear')
            left_decomp = tf.expand_dims(tf.concat([left_pos, left_neg], axis=-1), -1)
            right_decomp = tf.expand_dims(tf.concat([right_pos, right_neg], axis=-1), -1)



        with tf.name_scope('Composition'):
            sims = []
            # pooled_left = []
            # pooled_right = []
            for i, filter_size in enumerate(self.filter_sizes):
                left_conv = self.convolution(left_decomp, 2 * self.embedding_size, filter_size, 70, 'conv_left')
                left_pooled = self.max_pool(left_conv, filter_size, 'max_pooling_left')
                left_pooled_flatten = tf.reshape(left_pooled, [-1, 70], name='left_pooled_flatten')

                right_conv = self.convolution(right_decomp, 2 * self.embedding_size, filter_size, 70, 'conv_right')
                right_pooled = self.max_pool(right_conv, filter_size, 'max_pooling_right')
                right_pooled_flatten = tf.reshape(right_pooled, [-1, 70], name='right_pooled_flatten')
                
                sims.append(self.get_sim(left_pooled_flatten, right_pooled_flatten, 70, str(filter_size)))
                sims.append(self.get_cos_sim(left_pooled_flatten, right_pooled_flatten, str(filter_size)))
                # pooled_left.append(left_pooled_flatten)
                # pooled_right.append(right_pooled_flatten)

            sims = tf.concat(sims, axis=-1)
            # pooled_left = tf.concat(pooled_left, axis=-1)
            # pooled_right = tf.concat(pooled_right, axis=-1)


        with tf.variable_scope('output_layer'):
            self.output_features = sims
            self.output_features = tf.layers.batch_normalization(self.output_features,training=self.training,name='fea_bn_')
            self.output_features = tf.nn.dropout(self.output_features, self.keep_prob, name='hidden_output_drop')
            self.fc = tf.contrib.layers.fully_connected(
                inputs = self.output_features,
                num_outputs= self.fc_hidden_size,
                weights_initializer=tf.contrib.layers.xavier_initializer(),
                biases_initializer=tf.constant_initializer(1e-02),
                scope="FC"
            )
            self.fc_bn = tf.layers.batch_normalization(self.fc,training=self.training,name='fc_bn_')
            self.fc_bn = tf.nn.relu(self.fc_bn)
            self.hidden_drop = tf.nn.dropout(self.fc_bn, self.keep_prob, name='fc_drop')

        
        with tf.variable_scope('out_layer'):
            sim = tf.contrib.layers.fully_connected(
                inputs = self.hidden_drop,
                num_outputs= self.n_class,
                activation_fn = None,
                weights_initializer=tf.contrib.layers.xavier_initializer(),
                biases_initializer=tf.constant_initializer(1e-04),
                scope="FC_2"
            )
        return sim



    def get_cos_sim(self, x1, x2, name_scope):
        with tf.variable_scope('cos_sim_' + name_scope):
            norm1 = tf.sqrt(tf.reduce_sum(tf.square(x1), axis=1))
            norm2 = tf.sqrt(tf.reduce_sum(tf.square(x2), axis=1))
            dot_products = tf.reduce_sum(x1 * x2, axis=1, name='cos_sim')

            cos_sim = tf.expand_dims(dot_products / (norm1 * norm2), -1)
            return cos_sim


    def get_sim(self, x1, x2, d, name_scope):
        with tf.variable_scope("similarity_" + name_scope):
            M = tf.get_variable(
                name = 'M',
                shape = [d, d],
                initializer = tf.contrib.layers.xavier_initializer()
            )

            x1_trans = tf.matmul(x1, M)
            sims = tf.reduce_sum(tf.multiply(x1_trans, x2), axis=1, keepdims=True)
            return sims

    def max_pool(self, x, filter_size, name_scope):
        with tf.name_scope(name_scope + str(filter_size)):
            pool_width = 20 - filter_size + 1

            pooled = tf.layers.max_pooling2d(
                inputs=x,
                pool_size=(pool_width, 1),
                strides=1,
                padding='VALID',
            )
            # [batch_size, 1, 1, n_filter]
            return pooled

    def convolution(self, x, d, filter_size, n_filter, name_scope):
        with tf.name_scope(name_scope + str(filter_size)):
            conv = tf.contrib.layers.conv2d(
                inputs=x,
                num_outputs=n_filter,
                kernel_size=(filter_size, d),
                stride=1,
                padding='VALID',
                activation_fn=tf.nn.relu,
                weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
                biases_initializer=tf.constant_initializer(0.1),
                trainable=True
            )
            # [batch_size, |s| - filter_size + 1, 1, n_filter]
            return conv

    def decompose(self, X, X_hat, scope, type='linear'):
        with tf.variable_scope('decompose_'+scope):
            if type == 'rigid':
                mask_pos = tf.cast(tf.equal(X, X_hat), dtype='float32')
                X_pos = X * mask_pos
                mask_neg = tf.cast(tf.equal(mask_pos, 0.0), dtype='float32')
                X_neg = X * mask_neg
            elif type == 'linear':
                denom = (tf.linalg.norm(X, axis=-1, keepdims=True)) * (tf.linalg.norm(X_hat, axis=-1, keepdims=True))
                alpha = tf.reduce_sum(X * X_hat, axis=-1, keepdims=True) / denom
                X_pos = alpha * X
                X_neg = (1 - alpha) * X
            elif type == 'orthogonal':
                denom = tf.reduce_sum(X_hat * X_hat, axis=-1, keepdims=True)
                X_pos = tf.reduce_sum(X * X_hat, axis=-1, keepdims=True) / denom  * X_hat
                X_neg = X - X_pos
            return X_pos, X_neg

    def semantic_match(self, X, A, w, scope, type='local_w'):
        # x: batch * length2 * embedding_size
        # A: batch * length1 * length2
        # return: batch * length * embedding_size
        # type == 'max'只需将w设置为0， 'global'只需将w设置为sentence_length
        with tf.variable_scope('semantic_' + scope):
            pivot = tf.argmax(A, axis=-1)
            if type == 'global':
                return tf.matmul(A, X)/tf.expand_dims(tf.reduce_sum(A, axis=-1), axis=-1)
            elif type == 'local_w':
                upper = tf.expand_dims(tf.minimum(pivot + w, 20),-1)
                lower = tf.expand_dims(tf.maximum(pivot - w, 0), -1)
                indices = tf.tile(tf.expand_dims(tf.range(start=0, limit=20, dtype='int64'), axis=0), [50, 1])
                print(indices,"*********************",lower)
                mask = tf.cast(((indices>=lower)&(indices<=upper)), 'float32')
                return tf.matmul(A*mask, X)/tf.expand_dims(tf.reduce_sum(A*mask, axis=-1), axis=-1)

    def make_word_sim(self, x, y):
        dot = tf.einsum('abd,acd->abc', x, y)
        norm1 = tf.sqrt(tf.reduce_sum(tf.square(x), axis=2, keepdims=True))
        norm2 = tf.sqrt(tf.reduce_sum(tf.square(y), axis=2, keepdims=True))
        return dot / tf.einsum('abd,acd->abc', norm1, norm2)
    


    def cnn_inference(self, X_inputs, n_step,name):
        """TextCNN 模型。
        Args:
            X_inputs: tensor.shape=(batch_size, n_step)
        Returns:
            title_outputs: tensor.shape=(batch_size, self.n_filter_total)
        """
        inputs = X_inputs
        inputs = tf.expand_dims(inputs, -1)
        pooled_outputs = list()
        for i, filter_size in enumerate(self.filter_sizes):
            with tf.variable_scope(("conv-maxpool-%s" % filter_size)+name):
                # Convolution Layer
                filter_shape = [filter_size, self.embedding_size, 1, self.n_filter]
                W_filter = self.weight_variable(shape=filter_shape, name='W_filter')
                beta = self.bias_variable(shape=[self.n_filter], name='beta_filter')
                conv = tf.nn.conv2d(inputs, W_filter, strides=[1, 1, 1, 1], padding="VALID", name="conv")
                conv_bn=tf.layers.batch_normalization(conv,training=self.training,name=name)
                # Apply nonlinearity, batch norm scaling is not useful with relus
                # batch norm offsets are used instead of biases,使用 BN 层的 offset，不要 biases
                h = tf.nn.relu(conv_bn, name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(h, ksize=[1, n_step - filter_size + 1, 1, 1],
                                        strides=[1, 1, 1, 1], padding='VALID', name="pool")
                pooled_outputs.append(pooled)
        h_pool = tf.concat(pooled_outputs, 3)
        h_pool_flat = tf.reshape(h_pool, [-1, self.n_filter_total])
        return h_pool_flat  # shape = [batch_size, self.n_filter_total]



def loader(data_store_path=''):############
    # all_data=pd.read_csv(data_store_path+"data_sets/train_feature_second_no_query_and_title_data.csv")
    # query_title=pd.read_csv(data_store_path+"data_sets/train_data.csv",usecols=['query','title'])

    print("开始加载数据，有ctr")
    # train_data=pd.read_pickle(data_store_path+"train_feature_first_no_query_and_title_data.pickle")[train_start_index:train_end_index].reset_index(drop=True)
    query_title=pd.read_pickle("/home/kesci/test_finall_all_parts/part_0.pickle")
    
    query_title=pd.concat([query_title,pd.DataFrame(np.zeros([5000000,1]))],axis=1)
    
    print("加载数据完成")
    return query_title,1

def batch_iter(data_x,data_y, batch_size, num_epochs, shuffle=False):#这个就是产生batch的，可以直接用
    """
    Generates a batch iterator for a dataset.
    """
    # data_x=data_x.tolist()
    data=list(zip(data_x,data_y))
    data=np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:#如果为true 则代表允许随机取数据
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]#随机取数据
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]#而不用一个个加到列表里了。这是一个batch，嗯哼
            #取到的维度为（batchsize,2） y为(batchsize,1) 里面存的是序号

from gensim import models
import pickle
fw=open('dictionary/dictionary.pickle','rb')
dictionary=pickle.load(fw)
import pickle
fw=open('tfidf_model/tfidf_model.pickle','rb')
tfidf_model=pickle.load(fw)

def get_w2v_array(word_list,max_len):
    single_sentence_tfidf=tfidf_model[dictionary.doc2bow(word_list)]
    # single_sentence_tfidf={dictionary[word_id_tfidf[0]]:word_id_tfidf[1] for word_id_tfidf in single_sentence_tfidf}
    array = np.zeros((max_len, 100))
    if len(word_list)<=max_len:
        for i in range(len(word_list)):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
                # try:
                #     array[i][:]*=single_sentence_tfidf[str(word_list[i])]

                # except:
                #     pass
                
    else:
        for i in range(max_len):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
                # try:
                #     array[i][:]*=single_sentence_tfidf[str(word_list[i])]
                # except:
                #     pass
    return array

settings = Settings()

train_batches,vaild_data=loader()

train_batches=np.array(train_batches)
train_batches_y=train_batches[:,2]
train_batches_x=np.delete(train_batches, 2, axis=1)######这里是除了label以外的维数


batches=batch_iter(train_batches_x,train_batches_y,settings.train_batch_size,1)


print("train starting")
w2v_model = Word2Vec.load('w2v_model3/w2v_all_data_model.txt')
word_wv= w2v_model.wv


settings = Settings()

settings.all_features_num=train_batches_x.shape[1]-2###########自动设置数值型特征数量
model = TEXTCNN( settings)

learning_rate = tf.train.exponential_decay(settings.lr, model.global_step, settings.decay_step,
                                       settings.decay_rate, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate)

update_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    grads_and_vars=optimizer.compute_gradients(model.loss)
    train_op=optimizer.apply_gradients(grads_and_vars,global_step=model.global_step)

config = tf.ConfigProto() 
config.gpu_options.allow_growth = True 
sess = tf.Session(config=config) #启动创建的模型
# sess.run(tf.initialize_all_variables())

# saver = tf.train.Saver(tf.global_variables())
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
saver.restore(sess, tf.train.latest_checkpoint('/home/kesci/work/tf_model_textcnn_lcnn/'))
per_train_loss_avg_list=[]
per_vaild_loss_avg_list=[]
print("start")


parts_time=[]
from time import time
result_list=[]
for step,batch in enumerate(batches):
    time1=time()
    batch_x,batch_y=zip(*batch)
    batch_y=np.array(batch_y)[:,None]
    batch_x=[i.tolist() for i in batch_x]
    batch_x=pd.DataFrame(batch_x)
    batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
    batch_x=np.array(batch_x)
    #####[None,title_len,embedding_size]
    #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
    #query   title   title_count         title_ctr      label
    batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=batch_x)
    batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=batch_x)
    batch_x_ctr_fea=batch_x[:,2:]
    
    #     train_op=optimizer.apply_gradients(grads_and_vars,global_step=global_step)                              
    feed_dict = {model.embedded_x_query: batch_x_query,
                 model.embedded_x_title: batch_x_title,
                 model.batch_size: batch_x.shape[0], 
                 model.training: False, 
                 model.keep_prob: 1,}

    result=sess.run([model._y_pred],feed_dict)
    result_list.append(result)
    time2=time()
    parts_time.append(time2-time1)
    if step%1000==0:
        print(result[0][0])  
    if step%1000==0:  
        time_single_part=sum(parts_time)
        ed_time=(20000000/settings.train_batch_size-step)/100*time_single_part
        print("距预测完一整轮剩余时间：",ed_time/60," 分钟")
        parts_time=[]
    
    


In [2]:
for i in range(19):
    print(i)
    def loader(data_store_path=''):############
        # all_data=pd.read_csv(data_store_path+"data_sets/train_feature_second_no_query_and_title_data.csv")
        # query_title=pd.read_csv(data_store_path+"data_sets/train_data.csv",usecols=['query','title'])
    
        print("开始加载数据，有ctr")
        # train_data=pd.read_pickle(data_store_path+"train_feature_first_no_query_and_title_data.pickle")[train_start_index:train_end_index].reset_index(drop=True)
        query_title=pd.read_pickle("/home/kesci/test_finall_all_parts/part_"+str(i+1)+".pickle")
        
        query_title=pd.concat([query_title.reset_index(drop=True),pd.DataFrame(np.zeros([5000000,1]))],axis=1)
        
        print("加载数据完成")
        return query_title,1
    
    
    train_batches,vaild_data=loader()
    
    train_batches=np.array(train_batches)
    train_batches_y=train_batches[:,2]
    train_batches_x=np.delete(train_batches, 2, axis=1)######这里是除了label以外的维数
    
    
    batches=batch_iter(train_batches_x,train_batches_y,settings.train_batch_size,1)


    
    parts_time=[]
    from time import time

    for step,batch in enumerate(batches):
        time1=time()
        batch_x,batch_y=zip(*batch)
        batch_y=np.array(batch_y)[:,None]
        batch_x=[i.tolist() for i in batch_x]
        batch_x=pd.DataFrame(batch_x)
        batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
        batch_x=np.array(batch_x)
        #####[None,title_len,embedding_size]
        #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
        #query   title   title_count         title_ctr      label
        batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=batch_x)
        batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=batch_x)
        batch_x_ctr_fea=batch_x[:,2:]
        
        #     train_op=optimizer.apply_gradients(grads_and_vars,global_step=global_step)                              
        feed_dict = {model.embedded_x_query: batch_x_query,
                     model.embedded_x_title: batch_x_title,
                     model.batch_size: batch_x.shape[0], 
                     model.training: False, 
                     model.keep_prob: 1,}
    
        result=sess.run([model._y_pred],feed_dict)
        result_list.append(result)
        time2=time()
        parts_time.append(time2-time1)
        if step%1000==0:
            print(result[0][0])  
        if step%1000==0:  
            time_single_part=sum(parts_time)
            ed_time=(20000000/settings.train_batch_size-step)/100*time_single_part
            print("距预测完一整轮剩余时间：",ed_time/60," 分钟")
            parts_time=[]
    
    

In [5]:
result_list2=[pd.DataFrame(i[0]) for i in result_list]
y_pred=pd.concat(result_list2).reset_index(drop=True)
y_pred

In [6]:
y_pred.mean()

In [7]:
y_pred.to_pickle("/home/kesci/test_final_result/lcnn_fea_chunwenben.pickle")
y_pred.to_pickle("/home/kesci/work/test_final_result/lcnn_fea_chunwenben.pickle")